In [6]:
import json
with open('config.json','r') as f:
    slack_token = json.load(f)["SLACK_TOKEN"]


In [7]:
def get_tmp_folder():
    SAVEFOLDER = "/tmp"
    import platform
    pf = platform.system()
    if pf == 'Windows':
        if not os.path.exists("save_folder"):
            os.mkdir("save_folder")
        SAVEFOLDER = 'save_folder'
    elif pf == 'Darwin':
        SAVEFOLDER = "/tmp"
    elif pf == 'Linux':
        SAVEFOLDER = "/tmp"
    return SAVEFOLDER



In [8]:
import logging
logging.basicConfig()
import os
import time
import pprint

from boxsdk import JWTAuth, Client
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from logging import getLogger, StreamHandler, DEBUG
# チャンネル一覧を取得する
# slack_token = os.environ.get('SLACK_TOKEN')
logger = getLogger(__name__)
client = WebClient(token=slack_token)

SAVEFOLDER = get_tmp_folder()


In [59]:
TERM=0
channel_id = 'C03HFFXSGUA'
latest = int(time.time() - TERM)  # 現在日時 - 2週間 の UNIX時間
cursor = None  # シーク位置。最初は None ページを指定して、次からは next_cursor が指し示す位置。


try:
    response = client.conversations_history(  # conversations_history ＝ チャット一覧を得る
        channel=channel_id,
        latest=latest,
        cursor=cursor  # チャンネルID、latest、シーク位置を指定。
        # latestに指定した時間よりも古いメッセージが得られる。latestはUNIX時間で指定する。
    )
except SlackApiError as e:
    exit

In [45]:
file_ids = []
bot_user_id='U03H0TMS9FH'
for page in client.files_list(user=bot_user_id):
    for file in page.get("files", []):
        info = {
                'file_id' : file["id"],
                'channel_id' : file['channels'],
                "file_name" : file["name"],
                "user" : file["user"],
                "download_url" : file["url_private_download"],
                "timestamp" : file["timestamp"]
             }
        file_ids.append(info)
        
# pprint.pprint(file)
file['channels'][0]
pprint.pprint(file['channels'])

['C03L80LQYT0']


In [50]:
#どこにも保存されていないファイルがある
len(file_ids[0]['channel_id'])

0

In [13]:
res = client.files_list(user=bot_user_id)

{'ok': True, 'query': '*', 'files': {'total': 1, 'pagination': {'total_count': 1, 'page': 1, 'per_page': 20, 'page_count': 1, 'first': 1, 'last': 1}, 'paging': {'count': 20, 'total': 1, 'page': 1, 'pages': 1}, 'matches': [{'id': 'F03L6DU0SMC', 'created': 1655735793, 'timestamp': 1655735793, 'name': 'sample.png', 'title': 'sample.png', 'mimetype': 'image/png', 'filetype': 'png', 'pretty_type': 'PNG', 'user': 'U03H0TMS9FH', 'editable': False, 'size': 9103, 'mode': 'hosted', 'is_external': False, 'external_type': '', 'is_public': True, 'public_url_shared': False, 'display_as_bot': False, 'username': '', 'url_private': 'https://files.slack.com/files-pri/T03HFFHAWDQ-F03L6DU0SMC/sample.png', 'url_private_download': 'https://files.slack.com/files-pri/T03HFFHAWDQ-F03L6DU0SMC/download/sample.png', 'media_display_type': 'unknown', 'thumb_64': 'https://files.slack.com/files-tmb/T03HFFHAWDQ-F03L6DU0SMC-ebd632b300/sample_64.png', 'thumb_80': 'https://files.slack.com/files-tmb/T03HFFHAWDQ-F03L6DU0SMC-ebd632b300/sample_80.png', 'thumb_360': 'https://files.slack.com/files-tmb/T03HFFHAWDQ-F03L6DU0SMC-ebd632b300/sample_360.png', 'thumb_360_w': 360, 'thumb_360_h': 180, 'thumb_480': 'https://files.slack.com/files-tmb/T03HFFHAWDQ-F03L6DU0SMC-ebd632b300/sample_480.png', 'thumb_480_w': 480, 'thumb_480_h': 240, 'thumb_160': 'https://files.slack.com/files-tmb/T03HFFHAWDQ-F03L6DU0SMC-ebd632b300/sample_160.png', 'thumb_720': 'https://files.slack.com/files-tmb/T03HFFHAWDQ-F03L6DU0SMC-ebd632b300/sample_720.png', 'thumb_720_w': 720, 'thumb_720_h': 360, 'thumb_800': 'https://files.slack.com/files-tmb/T03HFFHAWDQ-F03L6DU0SMC-ebd632b300/sample_800.png', 'thumb_800_w': 800, 'thumb_800_h': 400, 'original_w': 800, 'original_h': 400, 'thumb_tiny': 'AwAYADDS7U3n1p3YUgz6UAAPFLn3o/CloASilooAaMn0o2/5xSr0P1o7UAN/L8qAP84paKADIpaSloA//9k=', 'permalink': 'https://slackdevtestenv.slack.com/files/U03H0TMS9FH/F03L6DU0SMC/sample.png', 'permalink_public': 'https://slack-files.com/T03HFFHAWDQ-F03L6DU0SMC-b54301f2bf', 'shares': {'public': {'C03HFFXSGUA': [{'ts': '1655735801.003739', 'channel_name': 'testchannel', 'team_id': 'T03HFFHAWDQ', 'share_user_id': 'U03H0TMS9FH'}]}}, 'channels': ['C03HFFXSGUA'], 'groups': [], 'ims': [], 'has_rich_preview': False}]}}

In [43]:
# response['ok'] # trueでok
# response['files']['total'] #1以上でok
# matched_files = response['files']['matches']
# i = 0
# filename = matched_files[i]['name']
# user = matched_files[i]['user']
# download_url = matched_files[i]['url_private_download']
# timestamp = matched_files[i]['timestamp']
# # timestamp = matched_files[i]['channels']
# response['files']['channels']
# # from datetime import datetime
# # datetime.fromtimestamp(timestamp)

KeyError: 'channels'

# 対象はSlackの指定する3チャンネルにアップロードされている画像(および動画)
# チャンネル一覧を取得する
# fileの一覧を取得する
# {name, upload_date, channel_name(or id)}
# Boxにファイルがあるかを確認する
# ファイルをダウンロード
# 


- Box内のチャンネル名フォルダ配下に日付フォルダを作成し、
- 作成したフォルダにアップロードする
- Slackの容量がいっぱいにならない事を目的に、定期的に実行する
- 一定期間経過したファイルを対象にする(初期値は1か月経過)
- Pythonの自動実行環境の構築はクラウド上に行う想定

In [52]:
#ファイルのダウンロード
import requests
import codecs
for file in file_ids:
    file_url = file["download_url"]
    content = requests.get(
                file_url,
                allow_redirects=True,
                headers={'Authorization': f'Bearer {slack_token}'},
                stream=True
            ).content
    save_path = get_tmp_folder() + '/' + file["file_name"]
    target_file = codecs.open(save_path, 'wb')
    target_file.write(content)
    target_file.close()
        